# Tarefa 1 — AdaBoost

## 1) Cinco diferenças entre Random Forest e AdaBoost

1. **Estratégia do ensemble**
   - **Random Forest (RF):** usa *bagging* (amostragem bootstrap + média/votação).
   - **AdaBoost:** usa *boosting* (modelos em sequência, cada novo modelo tenta corrigir erros do anterior).

2. **Treinamento dos modelos**
   - **RF:** árvores são treinadas **em paralelo** e de forma independente.
   - **AdaBoost:** modelos são treinados **em série** (um depende do resultado do anterior).

3. **Peso das amostras**
   - **RF:** amostras tendem a ter “peso igual” (o bootstrap muda a amostra por sorteio).
   - **AdaBoost:** aumenta o **peso** das amostras erradas para forçar o próximo modelo a focar nelas.

4. **Seleção de variáveis (features)**
   - **RF:** em cada split, a árvore avalia um **subconjunto aleatório de features** (`max_features`), aumentando diversidade.
   - **AdaBoost:** a diversidade vem principalmente do **re-peso das amostras** e do `learning_rate` (não depende de `max_features` como pilar principal).

5. **Robustez a ruído/outliers**
   - **RF:** costuma ser mais robusto a ruído e outliers.
   - **AdaBoost:** pode ser mais sensível a ruído/outliers, pois tende a “perseguir” exemplos difíceis (reponderados).

---

## 2) O que é AdaBoost (resumo do método)

AdaBoost (*Adaptive Boosting*) é um método de ensemble que combina vários modelos fracos (geralmente *stumps*, árvores rasas) para formar um modelo forte.  
A ideia é simples: treinamos um primeiro modelo, avaliamos onde ele erra e, na próxima rodada, aumentamos a importância (peso) das amostras erradas. Repetimos isso por várias rodadas e, no final, combinamos os modelos com pesos (modelos que performam melhor tendem a “contar mais” na decisão final).

---

## 3) Exemplo do AdaBoost em Python (baseado no exemplo do Scikit-learn)

Nesta parte, foi utilizado o dataset `load_iris`, com separação treino/teste, treino do `AdaBoostClassifier` (com estimador base simples) e avaliação por métricas como acurácia, relatório de classificação e matriz de confusão.

---

## 4) Cinco hiperparâmetros importantes no AdaBoost

1. **`n_estimators`**  
   Quantidade de modelos (rodadas de boosting). Mais modelos podem melhorar desempenho, mas aumentam custo e risco de overfitting.

2. **`learning_rate`**  
   Controla o “tamanho do passo” do boosting. Valores menores reduzem a contribuição de cada modelo e podem exigir mais estimators.

3. **`estimator` (ou `base_estimator`, dependendo da versão)**  
   Define o modelo fraco usado em cada rodada (comum: `DecisionTreeClassifier(max_depth=1)`).

4. **`algorithm`** *(quando disponível / dependendo da versão)*  
   Estratégia do boosting (ex.: SAMME, SAMME.R). Afeta como os pesos e probabilidades são combinados.

5. **`random_state`**  
   Garante reprodutibilidade (muito útil para comparar resultados e entregar a atividade com consistência).

> **Observação:** além desses, os hiperparâmetros do estimador base (ex.: `max_depth`, `min_samples_leaf`) também impactam bastante.

---

## 5) GridSearch para melhores hiperparâmetros (load_iris) *(opcional)*

Como extra, foi aplicado `GridSearchCV` para testar combinações de hiperparâmetros (ex.: `n_estimators`, `learning_rate` e parâmetros do estimador base), retornando `best_params_`, `best_score_` (validação cruzada) e a performance final no conjunto de teste.

## 6) Exemplo do AdaBoost em Python (baseado no exemplo do Scikit-learn)

### 6.1 Importações


In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## 6.2 Carregar base (load_iris) e visualizar

In [2]:
data = load_iris(as_frame=True)

df = data.frame.copy()
print("Shape do df:", df.shape)
print("Colunas:", df.columns.tolist())

df.head()

Shape do df: (150, 5)
Colunas: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)', 'target']


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


## 6.3 Separar X e y + train/test split

In [3]:
target = "target"
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print("Treino:", X_train.shape, "Teste:", X_test.shape)

Treino: (105, 4) Teste: (45, 4)


## 6.4 Treinar AdaBoost

Boas práticas: usar um estimador fraco como base (ex.: árvore rasa).

In [4]:
base = DecisionTreeClassifier(max_depth=1, random_state=42)

adb = AdaBoostClassifier(
    estimator=base,
    n_estimators=200,
    learning_rate=0.5,
    random_state=42
)

adb.fit(X_train, y_train)
y_pred = adb.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Acurácia (AdaBoost): {acc:.4f}")

Acurácia (AdaBoost): 0.9333


## 6.5 Relatório e Matriz de Confusão

In [5]:
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nMatriz de confusão:\n", confusion_matrix(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.88      0.93      0.90        15
           2       0.93      0.87      0.90        15

    accuracy                           0.93        45
   macro avg       0.93      0.93      0.93        45
weighted avg       0.93      0.93      0.93        45


Matriz de confusão:
 [[15  0  0]
 [ 0 14  1]
 [ 0  2 13]]


## 6.6 Resumo (y_real vs y_pred)

In [6]:
res = pd.DataFrame({
    "y_real": y_test.reset_index(drop=True),
    "y_pred": pd.Series(y_pred)
})

print("Linhas no resumo:", len(res))
res.head()


Linhas no resumo: 45


,y_real,y_pred
0,2,2
1,1,1
2,2,1
3,1,1
4,2,2


## 7) GridSearch para melhores hiperparâmetros (load_iris)

In [7]:
from sklearn.model_selection import GridSearchCV

base = DecisionTreeClassifier(random_state=42)

adb = AdaBoostClassifier(
    estimator=base,
    random_state=42
)

param_grid = {
    "n_estimators": [50, 100, 200, 400],
    "learning_rate": [0.01, 0.05, 0.1, 0.5, 1.0],
    "estimator__max_depth": [1, 2, 3]
}

gs = GridSearchCV(
    estimator=adb,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1
)

gs.fit(X_train, y_train)

print("Melhores parâmetros:", gs.best_params_)
print("Melhor score (CV):", gs.best_score_)

best_model = gs.best_estimator_
y_pred_gs = best_model.predict(X_test)

acc_gs = accuracy_score(y_test, y_pred_gs)
print(f"Acurácia no teste (melhor modelo): {acc_gs:.4f}")


Melhores parâmetros: {'estimator__max_depth': 1, 'learning_rate': 0.05, 'n_estimators': 400}
Melhor score (CV): 0.9619047619047618
Acurácia no teste (melhor modelo): 0.9333
